In [ ]:
import os

In [ ]:
!pip install pydrive                             # Package to use Google Drive API - not installed in Colab VM by default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('drive/My Drive/MinutiaeNet-master/CoarseNet/')

In [ ]:
#
#os.chdir('drive/My Drive/Colab Notebooks/LearnNet/CoarseNet')
os.getcwd()

'/content/drive/My Drive/MinutiaeNet-master/CoarseNet'

In [ ]:
#!ls 'drive/My Drive/MinutiaeNet-master/CoarseNet'

In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [ ]:
!pip install tensorflow==1.15

In [ ]:
#
#os.chdir('drive/My Drive/Colab Notebooks/LearnNet/CoarseNet')
os.getcwd()


'/content/drive/My Drive/MinutiaeNet-master/CoarseNet'

In [ ]:
from google.colab import auth                    # Other necessary packages
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

In [ ]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1z8QvnJ30NzwuJaGsZ9mWR4L4N3Vx15xA"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("MinutiaeNet_utils.py")          # Save the .py module file to Colab VM
import MinutiaeNet_utils

W0331 06:51:39.691164 140623119017856 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/usr/local/lib/python2.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
MinutiaeNet_utils.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_m

In [ ]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1KETV6nYsHUaLcM0ivaTZPclcB1_6OKM1"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("CoarseNet_utils.py")          # Save the .py module file to Colab VM
import CoarseNet_utils 

W0331 06:51:43.947047 140623119017856 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0331 06:51:43.948573 140623119017856 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0331 06:51:43.968961 140623119017856 deprecation.py:323] From CoarseNet_utils.py:323: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1d7uunRT4nc_n2h0SGtzHGTryLNRLCd6f"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("FineNet_model.py")          # Save the .py module file to Colab VM
import FineNet_model

In [ ]:
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1gZXLuxihxb-TEPOQrfFAHVITddck8yVQ"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("CoarseNet_model.py")          # Save the .py module file to Colab VM
import CoarseNet_model

In [ ]:
"""Code for FineNet in paper "Robust Minutiae Extractor: Integrating Deep Networks and Fingerprint Domain Knowledge" at ICB 2018
  https://arxiv.org/pdf/1712.09401.pdf

  If you use whole or partial function in this code, please cite paper:

  @inproceedings{Nguyen_MinutiaeNet,
    author    = {Dinh-Luan Nguyen and Kai Cao and Anil K. Jain},
    title     = {Robust Minutiae Extractor: Integrating Deep Networks and Fingerprint Domain Knowledge},
    booktitle = {The 11th International Conference on Biometrics, 2018},
    year      = {2018},
    }
"""

from __future__ import absolute_import
from __future__ import division

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

from datetime import datetime
from keras import backend as K

from MinutiaeNet_utils import *
from CoarseNet_utils import *
from CoarseNet_model import *



os.environ["CUDA_VISIBLE_DEVICES"] = '0'

config = K.tf.ConfigProto(gpu_options=K.tf.GPUOptions(allow_growth=True))
sess = K.tf.Session(config=config)
K.set_session(sess)

mode = 'inference'
#mode = 'deploy'

# Can use multiple folders for deploy, inference
deploy_set = ['../Dataset/CoarseNet_train/',]
inference_set = ['../Dataset/CoarseNet_test/',]


pretrain_dir = '../Models/CoarseNet.h5'
output_dir = './output_CoarseNet/'+datetime.now().strftime('%Y%m%d-%H%M%S')

FineNet_dir = 'drive/My Drive/MinutiaeNet-master/Models/FineNet.h5'

def main():
    if mode == 'deploy':
        output_dir = '../output_CoarseNet/deployResults/'+datetime.now().strftime('%Y%m%d-%H%M%S')
        logging = init_log(output_dir)
        for i, folder in enumerate(deploy_set):
            deploy_with_GT(folder, output_dir=output_dir, model_path=pretrain_dir, FineNet_path=FineNet_dir)
            # evaluate_training(model_dir=pretrain_dir, test_set=folder, logging=logging)
    elif mode == 'inference':
        output_dir = '../output_CoarseNet/inferenceResults/'+datetime.now().strftime('%Y%m%d-%H%M%S')
        logging = init_log(output_dir)
        for i, folder in enumerate(inference_set):
            inference(folder, output_dir=output_dir, model_path=pretrain_dir, FineNet_path=FineNet_dir, file_ext='.bmp',
                      isHavingFineNet=False)
    else:
        pass

if __name__ =='__main__':
    main()

../Dataset/CoarseNet_test/
0
0.5 0.45
1
0.5 0.45
2
0.5 0.45
3
0.5 0.45
4
0.5 0.45
5
0.5 0.45
6
0.5 0.45
7
0.5 0.45
8
0.5 0.45
9
0.5 0.45
10
0.5 0.45
11
0.5 0.45
12
0.5 0.45
13
0.5 0.45
14
0.5 0.45
15
0.5 0.45
16
0.45 0.4
17
0.5 0.45
18
0.5 0.45
19
0.5 0.45
20
0.25 0.2
21
0.5 0.45
22
0.5 0.45
23
0.45 0.4
24
0.5 0.45
25
0.5 0.45
26
0.5 0.45
27
0.15 0.1
28
0.5 0.45
29
0.5 0.45
30
0.35 0.3
31
0.35 0.3
32
0.5 0.45
33
0.45 0.4
34
0.5 0.45
35
0.5 0.45
36
0.5 0.45
37
0.5 0.45
38
0.5 0.45
39
0.5 0.45
40
0.5 0.45
41
0.45 0.4
42
0.5 0.45
43
0.5 0.45
44
0.5 0.45
45
0.35 0.3
46
0.5 0.45
47
0.5 0.45
48
0.5 0.45
49
0.5 0.45
50
0.5 0.45
51
0.5 0.45
52
0.5 0.45
53
0.5 0.45
54
0.5 0.45
55
0.5 0.45
56
0.5 0.45
57
0.5 0.45
58
0.5 0.45
59
0.5 0.45
60
0.5 0.45
61
0.5 0.45
62
0.5 0.45
63
0.5 0.45
64
0.5 0.45
65
0.5 0.45
66
0.5 0.45
67
0.5 0.45
68
0.5 0.45
69
0.5 0.45
70
0.5 0.45
71
0.5 0.45
72
0.5 0.45
73
0.5 0.45
74
0.5 0.45
75
0.3 0.25
76
0.35 0.3
77
0.5 0.45
78
0.5 0.45
79
0.5 0.45
